<a href="https://colab.research.google.com/github/ap3s-plan3t/itz-intr-det/blob/main/i_tzo_intr_det.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight


epochs = 100
nclass = 12

In [2]:
"""
Baseline models
"""

from tensorflow.keras.layers import Flatten,Dense,\
    Input,Dropout, Conv1D, AvgPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers,layers,losses


#------------------------------------------------------------------------------
# LSTM
#------------------------------------------------------------------------------

def model_lstm(lr=1e-4,N=64,inshape=40,nclass=12):
    in1=Input(shape=(inshape,1))
    x=layers.LSTM(N, activation='tanh')(in1)
    x=Dropout(0.1)(x)
    x=Dense(128, activation='relu')(x)
    out1=Dense(nclass, activation='softmax')(x)
    model=Model(inputs=in1,outputs=out1)

    model.compile(optimizer=optimizers.Adam(lr),
              loss=losses.categorical_crossentropy,
              metrics=['acc'])
    return model

#------------------------------------------------------------------------------
#CONV1D
#------------------------------------------------------------------------------

def model_conv1D(lr=1e-4,N=64,inshape=40,nclass=12):
    in1=Input(shape=(inshape,1))
    x=Conv1D(N, 3,padding='same',activation='relu')(in1)
    x=Conv1D(N, 3,padding='same',activation='relu')(x)

    x=layers.Flatten()(x)
    x=Dropout(0.1)(x)
    x=Dense(128, activation='relu')(x)
    out1=Dense(nclass, activation='softmax')(x)
    model=Model(inputs=in1,outputs=out1)

    model.compile(optimizer=optimizers.Adam(lr),
              loss=losses.categorical_crossentropy,
              metrics=['acc'])
    return model


#------------------------------------------------------------------------------
# DENSE
#------------------------------------------------------------------------------

def model_dense(lr=1e-4,N=64,inshape=40,nclass=12):
    in1=Input(shape=(inshape,1))
    x=Dense(N,activation='relu')(in1)
    x=Dense(N,activation='relu')(x)
    x=layers.Flatten()(x)
    x=Dropout(0.1)(x)
    x=Dense(128, activation='relu')(x)
    out1=Dense(nclass, activation='softmax')(x)
    model=Model(inputs=in1,outputs=out1)

    model.compile(optimizer=optimizers.Adam(lr),
              loss=losses.categorical_crossentropy,
              metrics=['acc'])
    return model

#------------------------------------------------------------------------------
# BASE LINE models
#------------------------------------------------------------------------------
def model_conv1D_large(nfeat=40,lr=1e-2,nclass=12):
    in1=Input(shape=(nfeat,1))
    x=Conv1D(64, 3,padding='same' ,activation='relu')(in1)
    x=Conv1D(64, 3,padding='same' ,activation='relu')(x)
    x=AvgPool1D()(x)
    x=Conv1D(128,5,padding='same' ,activation='relu')(x)
    x=AvgPool1D()(x)
    x=Conv1D(256,7,padding='same'  ,activation='relu')(x)
    x=AvgPool1D()(x)
    x=Conv1D(512,9,padding='same' ,activation='relu')(x)
    x=Flatten()(x)
    x=Dropout(0.4)(x)
    x=Dense(512, activation='relu')(x)
    output=Dense(nclass, activation='softmax')(x)
    model = Model(inputs=in1, outputs=output)

    #opt=optimizers.SGD(lr)
    opt=optimizers.Adam(lr)
    # opt=optimizers.RMSprop(lr)
    # opt=optimizers.Adam(lr)
    #opt=optimizers.Nadam(2e-2)
    model.compile(optimizer=opt,#Adam(lr=1e-2),
              loss=losses.categorical_crossentropy,
              metrics=['acc'])
    return model

def model_conv1D_binary(nfeat=32,lr=1e-2,nclass=12):
    in1=Input(shape=(nfeat,1))
    x=Conv1D(64, 3,padding='same' ,activation='relu')(in1)
    x=Conv1D(64, 3,padding='same' ,activation='relu')(x)
    x=AvgPool1D()(x)
    x=Conv1D(128,5,padding='same' ,activation='relu')(x)
    x=AvgPool1D()(x)
    x=Conv1D(256,7,padding='same'  ,activation='relu')(x)
    x=AvgPool1D()(x)
    x=Conv1D(512,9,padding='same' ,activation='relu')(x)
    x=Flatten()(x)
    x=Dropout(0.2)(x)
    x=Dense(512, activation='relu')(x)
    output=Dense(nclass, activation='sigmoid')(x)
    model = Model(inputs=in1, outputs=output)

    #opt=optimizers.SGD(lr)
    # opt=optimizers.Adam(lr)
    opt=optimizers.RMSprop(lr)
    # opt=optimizers.Adam(lr)
    #opt=optimizers.Nadam(2e-2)
    model.compile(optimizer=opt,#Adam(lr=1e-2),
              loss=losses.binary_crossentropy,
              metrics=['acc'])
    return model

In [8]:
def loadDataset():
            test_data.astype('float32'), test_label.astype('int32')

In [9]:
# -- LOAD DATA -----------------------------------------------------------------
train_data, train_labelp, val_data, val_labelp, test_data, test_labelp = loadDataset()

NameError: name 'test_data' is not defined

In [10]:
# to_categorical
train_label = to_categorical(train_labelp, nclass)
val_label   = to_categorical(val_labelp,   nclass)
test_label  = to_categorical(test_labelp,  nclass)

print('train_data.shape=', train_data.shape)
print('test_data.shape=',  test_data.shape)
print('test_data.shape=',  val_data.shape)

NameError: name 'train_labelp' is not defined

In [13]:
#get the number of features
inshape=train_data.shape[1]

NameError: name 'train_data' is not defined

In [14]:
# Class balancing weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(
                                                      train_labelp),
                                                  y=train_labelp)


class_weights = {i: class_weights[i] for i in range(len(class_weights))}

NameError: name 'train_labelp' is not defined

In [15]:
# -- CALLBACKS -----------------------------------------------------------------
earlyStopping = EarlyStopping(monitor='val_loss',
                              patience=30,
                              verbose=0,
                              mode='min')

modelCheckPoint = ModelCheckpoint('./savemodels/model5class.weights.{epoch:03d}-{val_acc:.4f}.hdf5',
                                  save_best_only=True,
                                  monitor='val_acc',
                                  mode='max')

In [16]:
# reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss',
#                                   factor=0.1,
#                                   patience=7,
#                                   verbose=1,
#                                   epsilon=1e-4,
#                                   mode='min')

# -- Baseline models-----------------------------------------------------------

# -- Conv1d
model=models_ddos.model_conv1D(lr=1e-4,N=64,inshape=inshape)
# -- Dense
# model=models_ddos.model_dense(lr=1e-4,N=64,inshape=inshape)
# -- LSTM
# model=models_ddos.model_lstm(lr=1e-4,N=64,inshape=inshape)

model.summary()

NameError: name 'models_ddos' is not defined

In [ ]:
# -----------------------------------------------------------------------------
# print model to an image file
# dot_img_file = 'model1.png'
# plot_model(model, to_file=dot_img_file, show_shapes=True)

In [ ]:
# -- TRAIN MODEL --------------------------------------------------------------
history = model.fit(train_data,
                    train_label,
                    shuffle=True,
                    epochs=epochs,
                    batch_size=256,  # 256,#128,#32, 64
                    # validation_data=validation_generator,
                    # validation_split=0.2,
                    # validation_data=(val_data,val_label),
                    validation_data=(val_data, val_label),
                    callbacks=[modelCheckPoint],
                    class_weight=class_weights,
                    workers=3)

In [ ]:
# -- Load best model ----------------------------------------------------------
str_models = os.listdir('./savemodels')
str_models = np.sort(str_models)
best_model = str_models[str_models.size-1]
print('best_model=', best_model)
model.load_weights('./savemodels/'+best_model)

In [ ]:
# --Confusion matrix ----------------------------------------------------------
print('TEST DATA-Confusion matrix:')
pred = model.predict(test_data)
pred_y = pred.argmax(axis=-1)

cm = confusion_matrix(test_labelp.astype('int32'), pred_y)
print(cm)

print('Accuracy ratios for each class')
print('WebDDoS      =', cm[0, 0]/np.sum(cm[0, :]))
print('BENIGN       =', cm[1, 1]/np.sum(cm[1, :]))
print('UDP-lag      =', cm[2, 2]/np.sum(cm[2, :]))
print('DrDoS_NTP    =', cm[3, 3]/np.sum(cm[3, :]))
print('Syn          =', cm[4, 4]/np.sum(cm[4, :]))
print('DrDoS_SSDP   =', cm[5, 5]/np.sum(cm[5, :]))
print('DrDoS_UDP    =', cm[6, 6]/np.sum(cm[6, :]))
print('DrDoS_NetBIOS=', cm[7, 7]/np.sum(cm[7, :]))
print('DrDoS_MSSQL  =', cm[8, 8]/np.sum(cm[8, :]))
print('DrDoS_SNMP   =', cm[9, 9]/np.sum(cm[9, :]))
print('TFTP         =', cm[10,10]/np.sum(cm[10, :]))
print('DrDoS_DNS    =', cm[11,11]/np.sum(cm[11, :]))

In [ ]:
# -- Confusion matrix plot
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
label=np.array(["WebDDoS","BENIGN","UDP-lag","DrDoS_NTP","Syn ",
                "DrDoS_SSDP","DrDoS_UDP","DrDoS_NetBIOS","DrDoS_MSSQL",
                "DrDoS_SNMP","TFTP","DrDoS_DNS"])

cmo = ConfusionMatrixDisplay(cm,display_labels=label)
fig, ax = plt.subplots(figsize=(12,12))
cmo.plot(ax=ax, xticks_rotation=45)

In [ ]:
# Plot training and validation accurry and loss graphs
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

np.save('historydata.npy',[acc,val_acc,loss,val_loss])
[acc, val_acc, loss, val_loss] = np.load('historydata.npy')

plt.figure()
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r.', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r.', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()